In [34]:
import os
import re
import json
from datetime import datetime
from math import ceil
from io import StringIO
# quant analysis
import sqlite3
import numpy as np
import pandas as pd
import statsmodels.api as sm
# import sklearn.datasets
import scipy.stats
# data viz
import matplotlib.pyplot as plt
%matplotlib inline

In [36]:
def CONFIG():
    """"""
    pid = 'analysis-baseball'
    now = datetime.now()
    cwd = os.getcwd()
    data = os.path.join(cwd,'data')
    
    cfg = {
        'data': data,
        'lahman': os.path.join(data, 'lahmansbaseballdb.sqlite'),
        'eda': {
            'path': os.path.join(cwd,'eda'),
            'key': '{table_name}.md'
        }
    }
    
    return cfg

In [41]:
def EDA(df,f_out,n=10):
    """"""    
    name = os.path.basename(f_out).split('.')[0]
    lb = '\n'
    lblb = '\n\n'
    
    # columns & types
    content = '# EDA - {} Files {}'.format(os.path.basename(name).upper(),lblb)
    content+='#### Column Name [IDX] -  Dtype (Head / Tail) {}'.format(lb)
    dtypes = df.dtypes.to_dict()
    head = df.head(1).T.iloc[:,0].to_list() # to_dict() - head.get(j)
    tail = df.tail(1).T.iloc[:,0].to_list() # to_dict() - tail.get(j)
    for i,j in enumerate(df.columns):
        content+='- **{}** [{}] - {} ({} / {}) {}'.format(j, i, dtypes.get(j), head[i], tail[i], lb)
    
    # html
    content+='{}#### Head / Tail [n={}] Sample {}'.format(lb+lblb,n,lblb)
    content+=(df.head(n).append(df.tail(n)).to_html(None,index=False,header=True))
    
    with open(f_out,'w') as f:
        f.write(content)

In [42]:
if __name__=='__main__':
    CFG = CONFIG()
    with sqlite3.connect(CFG['lahman']) as cnxn:
        query_base = 'select * from {table_name}'
#         query_base = "pragma table_info('{table_name}')"
        cur = cnxn.cursor()
        tables = [i[0] for i in cur.execute("select name from sqlite_master where type = 'table'").fetchall()]
        for table in tables:
            query = query_base.format(table_name=table)
            df = pd.read_sql(query,cnxn)
            EDA(df, os.path.join(CFG['eda']['path'],CFG['eda']['key'].format(table_name=table)))
            